In [50]:
import json
import pandas as pd
import uuid

In [6]:
#Jobname - SourceAttribute - SourceDatatype
df_Target_Parameters = pd.read_csv("C:\\Users\r.da.silva.severino.DIR\Downloads\SourceTargetParameters.csv", sep=';')
df_Target_Parameters.head()
# clis = []
# common_parameters = get_common_parameters()
# tables = get_tables(df_Target_Parameters)

,PathJavaScript,Jobname,PathJsonSchema,jsonFileName,CreationData,CreationOwner,TransformationType,SourceConnectionOrStorage,SourceTableOrFile,SourceAttribute,...,RelativePosition,SourceDatatype,SourceMode,TargetConnection,TargetTable,TargetAttribute,TargetDataType,TargetMode,TargetDescription,Transformation
0,scripts,MVW_BG_CLIENTE_JURIDICO_INCR.js,schemas,MVW_BG_CLIENTE_JURIDICO_INCR.json,NaN,NaN,file,gs://acn-agbg-academy-dataflow/data_files,MVW_BG_CLIENTE_JURIDICO_INCR.csv,PRT_PESSOA_JURIDICA_ID_CLIENTE,...,NaN,NUMBER,NULLABLE,BigQuery,acn-agbg-academy:SANPFPRD_BQ.MVW_BG_CLIENTE_JU...,PRT_PESSOA_JURIDICA_ID_CLIENTE,INTEGER,NULLABLE,Atributo utilizado para criar o relacionamento...,NaN
1,scripts,MVW_BG_CLIENTE_JURIDICO_INCR.js,schemas,MVW_BG_CLIENTE_JURIDICO_INCR.json,NaN,NaN,file,gs://acn-agbg-academy-dataflow/data_files,MVW_BG_CLIENTE_JURIDICO_INCR.csv,PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO,...,NaN,DATE,NULLABLE,BigQuery,acn-agbg-academy:SANPFPRD_BQ.MVW_BG_CLIENTE_JU...,PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO,DATETIME,NULLABLE,PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO,NaN
2,scripts,MVW_BG_CLIENTE_JURIDICO_INCR.js,schemas,MVW_BG_CLIENTE_JURIDICO_INCR.json,NaN,NaN,file,gs://acn-agbg-academy-dataflow/data_files,MVW_BG_CLIENTE_JURIDICO_INCR.csv,PRT_PESSOA_JURIDICA_TP_PESSOA_JURIDICA,...,NaN,VARCHAR2(1 BYTE),NULLABLE,BigQuery,acn-agbg-academy:SANPFPRD_BQ.MVW_BG_CLIENTE_JU...,PRT_PESSOA_JURIDICA_TP_PESSOA_JURIDICA,STRING,NULLABLE,"Tipo da pessoa juridica. P - Pessoa juridica, ...",NaN
3,scripts,MVW_BG_CLIENTE_JURIDICO_INCR.js,schemas,MVW_BG_CLIENTE_JURIDICO_INCR.json,NaN,NaN,file,gs://acn-agbg-academy-dataflow/data_files,MVW_BG_CLIENTE_JURIDICO_INCR.csv,PRT_PESSOA_JURIDICA_TP_EMPRESA,...,NaN,VARCHAR2(2 BYTE),NULLABLE,BigQuery,acn-agbg-academy:SANPFPRD_BQ.MVW_BG_CLIENTE_JU...,PRT_PESSOA_JURIDICA_TP_EMPRESA,STRING,NULLABLE,Atributo utilizado para criar o relacionamento...,NaN
4,scripts,MVW_BG_CLIENTE_JURIDICO_INCR.js,schemas,MVW_BG_CLIENTE_JURIDICO_INCR.json,NaN,NaN,file,gs://acn-agbg-academy-dataflow/data_files,MVW_BG_CLIENTE_JURIDICO_INCR.csv,PRT_PESSOA_JURIDICA_FG_DADO_CONFIRMADO,...,NaN,VARCHAR2(1 BYTE),NULLABLE,BigQuery,acn-agbg-academy:SANPFPRD_BQ.MVW_BG_CLIENTE_JU...,PRT_PESSOA_JURIDICA_FG_DADO_CONFIRMADO,STRING,NULLABLE,PRT_PESSOA_JURIDICA_FG_DADO_CONFIRMADO,NaN


In [63]:
def get_filename(table:str)->str:
    return f"message_body_{table}.json"

def get_tables(df_Target_Parameters:pd.DataFrame)->list[str]:

    return df_Target_Parameters["Jobname"].unique()

def get_fields(df_Target_Parameters:pd.DataFrame, tabela:str )->list[str]:
    filtro1 = df_Target_Parameters.loc[df_Target_Parameters['Jobname']==tabela]
    filtro1 = filtro1.set_index("SourceAttribute")
    return filtro1["SourceDatatype"].to_dict()

def get_cli(tabela:str, json_path:str):
    return f'gcloud scheduler jobs create http {tabela} --project="acn-agbg-academy" --location="us-central1" --schedule="0/15 * * * *" --uri="https://dataflow.googleapis.com//v1b3/projects/acn-agbg-academy/locations/us-central1/templates:launch?gcsPath=gs://acn-agbg-academy-dataflow/templates/JDBCToBigQuery_withTransformation.json" --time-zone="America/Sao_Paulo" --headers="User-Agent=Google-Cloud-Scheduler" --oauth-service-account-email="dataflow-templates-sa@acn-agbg-academy.iam.gserviceaccount.com" --message-body-from-file="{json_path}"'

def get_query(dataset:str, tabela:str, colunas_tipos:dict)->str:
    query = "SELECT "
    columns = colunas_tipos.keys()
    query_columns = []
    for column in columns:
        if 'DATE' in colunas_tipos[column] or 'TIMESTAMP' in colunas_tipos[column]:
            query_columns.append(f"TO_CHAR({column},'YYYY-MM-DD HH24:MI:SS') as {column}")
        else:
            query_columns.append(column)

    query = query + ", ".join(query_columns)
    query = query + f" FROM {dataset}.{tabela}"

    return query

def get_json_filename(table:str)->str:
    return f"body_message_{table}.json"

def get_message(dataset:str, table:str, query:str)->dict:
    message = {
        "jobName": f"jdbc_to_BQ_{table}_" + str(uuid.uuid4()),
        "parameters": {
            "driverJars": "gs://acn-agbg-academy-dataflow/templates/ojdbc8.jar",
            "driverClassName": "oracle.jdbc.driver.OracleDriver",
            "connectionURL": "jdbc:oracle:thin:@//172.22.61.51:1521/SANIMOBPRD",
            "connectionProperties": "",
            "username": "U_SANIMOBBDQ",
            "password": "sanimob#2022#big",
            "query": query,
            "JSONPath": f"gs://acn-agbg-academy-dataflow/schemas/{table}.json",
            "bigQueryLoadingTemporaryDirectory": "gs://acn-agbg-academy-dataflow/temp",
            "outputTable": f"acn-agbg-academy:{dataset}.{table}"
        },
        "environment": {
            "tempLocation": "gs://acn-agbg-academy-dataflow/temp",
            "subnetwork": "https://www.googleapis.com/compute/v1/projects/acn-agbg-academy/regions/us-central1/subnetworks/us-central1-subnet-b",
            "machineType": "n2-standard-2",
            "numWorkers": "1",
            "maxWorkers": "4",
            "ipConfiguration": "WORKER_IP_PRIVATE"
        }
    }

    return message

def write_message_file(json_path:str, message:dict):
    with open(json_path, "w") as f:
        json.dump(message,f)

def cria_bat(bat_path:str, clis:list[str]):
    with open(bat_path, "w") as f:
        f.write("\n".join(clis))
        

In [64]:
dataset = "SANPFPRD_BQ"
tables = get_tables(df_Target_Parameters)
clis = []
for table in tables:
    table_name = table.replace(".js", "")
    json_path = get_json_filename(table_name)
    cli = get_cli(table_name, json_path)#salvar lista
    colunas_tipos:dict = get_fields(df_Target_Parameters, table)
    query = get_query(dataset, table_name, colunas_tipos)
    
    message:dict = get_message(dataset, table_name, query)
    write_message_file(json_path, message)
    clis.append(cli)
cria_bat("schedule_http_jobs.bat", clis)

In [38]:
colunas_tipos = {'PRT_PESSOA_JURIDICA_ID_CLIENTE': 'NUMBER',
                'PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO': 'DATE',
                'PRT_PESSOA_JURIDICA_TP_PESSOA_JURIDICA': 'VARCHAR2(1 BYTE)',
                'PRT_PESSOA_JURIDICA_DH_ATUALIZACAO': 'TIMESTAMP (6)'
                }
tabela = "MVW_BG_CLIENTE_JURIDICO_INCR"
dataset = "SANPFPRD_BQ"
chaves = colunas_tipos.keys()


expected = "SELECT PRT_PESSOA_JURIDICA_ID_CLIENTE, TO_CHAR(PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO,'YYYY-MM-DD HH24:MI:SS') as PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO, PRT_PESSOA_JURIDICA_TP_PESSOA_JURIDICA, TO_CHAR(PRT_PESSOA_JURIDICA_DH_ATUALIZACAO,'YYYY-MM-DD HH24:MI:SS') as PRT_PESSOA_JURIDICA_DH_ATUALIZACAO FROM SANPFPRD_BQ.MVW_BG_CLIENTE_JURIDICO_INCR"
current = get_query(dataset, tabela, colunas_tipos)
assert(expected==current)
print(expected)
print(current)

SELECT PRT_PESSOA_JURIDICA_ID_CLIENTE, TO_CHAR(PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO,'YYYY-MM-DD HH24:MI:SS') as PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO, PRT_PESSOA_JURIDICA_TP_PESSOA_JURIDICA, TO_CHAR(PRT_PESSOA_JURIDICA_DH_ATUALIZACAO,'YYYY-MM-DD HH24:MI:SS') as PRT_PESSOA_JURIDICA_DH_ATUALIZACAO FROM SANPFPRD_BQ.MVW_BG_CLIENTE_JURIDICO_INCR
SELECT PRT_PESSOA_JURIDICA_ID_CLIENTE, TO_CHAR(PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO,'YYYY-MM-DD HH24:MI:SS') as PRT_PESSOA_JURIDICA_DT_ULT_ATUALIZACAO, PRT_PESSOA_JURIDICA_TP_PESSOA_JURIDICA, TO_CHAR(PRT_PESSOA_JURIDICA_DH_ATUALIZACAO,'YYYY-MM-DD HH24:MI:SS') as PRT_PESSOA_JURIDICA_DH_ATUALIZACAO FROM SANPFPRD_BQ.MVW_BG_CLIENTE_JURIDICO_INCR


In [ ]:

# df = pd.DataFrame({"JobName":["tabela1", "tabela1", "tabela2"], "Coluna12":["asd", "asd", "asd"]})
# tables = get_tables(df)
# assert(min(tables == ["tabela1", "tabela2"]))
    
# verificar SourceDatatype
# get_fields(df_Target_Parameters, 'MVW_BG_CLIENTE_JURIDICO_INCR.js')

# filename = get_filename('table1')
# assert(filename == "message_body_table1.json")
# get_cli('asfd', 'c:\\asdfa')